In [1]:
import pandas as pd

# = = =  = = =  = = =

from urllib.parse import urlparse, parse_qs, urlencode

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

import math

In [2]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel('./file/input.xlsx',
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：2



,No,Url
0,1,https://www.standardbrand.com/en/ecatalog?part...
1,2,https://www.standardbrand.com/en/ecatalog?part...


In [3]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                     1
 Url    https://www.standardbrand.com/en/ecatalog?part...
 Name: 0, dtype: object)

In [4]:
import time

url_parse = urlparse(crawler_series['Url'])

dict_param = parse_qs(url_parse.query)

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_request, url_check = f'''https://ecatalog.smpcorp.com/V2/STD/api/part/partsearch?filter={dict_param['part'][0]}&filterType={dict_param['search'][0] if 'search' in dict_param else 'n'}&searchType={dict_param['type'][0] if 'type' in dict_param else 'null'}&imageSize=80&start=0&limit=96&sort=3&catFilter=-All-&yearFilter=-All-&makeFilter=-All-&modelFilter=-All-&engineFilter=-All-&attrCodeFilter=-All-&attrValueFilter=-All-&plkEngineMakeFilter=-All-&plkEngineModelFilter=-All-&plkEngineDispFilter=-All-''', crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            list_dict = resp.json()

            count = list_dict[0]['maxRows']
            if count > 0:
                break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

count

1


450

In [5]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Url': crawler_series['Url'],
                          'SKU Count': count,
                          'Page': math.ceil(count / 96)})

list_dict_correct

[{'No': 1,
  'Url': 'https://www.standardbrand.com/en/ecatalog?part=Accelerator%20Pedal%20Sensor%20(APS)&type=p',
  'SKU Count': 450,
  'Page': 5}]

In [6]:
crawler_status = True

crawler_status

True

In [7]:
crawler_series['Request_Url'], crawler_series['Check_Url'] = url_request, url_check

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://www.standardbrand.com/en/ecatalog?part...
 Request_Url    https://ecatalog.smpcorp.com/V2/STD/api/part/p...
 Check_Url      https://www.standardbrand.com/en/ecatalog?part...
 Name: 0, dtype: object]

In [8]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel('./test_page.xlsx', index=False)

df_correct

Data Outputting...


,No,Url,SKU Count,Page
0,1,https://www.standardbrand.com/en/ecatalog?part...,450,5


In [9]:
print('Output Finished !')
print()

Output Finished !



In [10]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel('./test_page_error.xlsx', index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Url,Request_Url,Check_Url
0,1,https://www.standardbrand.com/en/ecatalog?part...,https://ecatalog.smpcorp.com/V2/STD/api/part/p...,https://www.standardbrand.com/en/ecatalog?part...
